# PX912: Solid Mechanics - Workshop 2

These workshops are not marked, but they should help you to better understand theoretical aspects presented in the lectures, which should help you prepare for the PX912 viva, and practice solutions with Python coding. The latter will also be used in your coursework project. 

### Please run the cell below!
This cell loads the core library written for this module. The core library contains hints, solution checking and grading. 

Make sure that the output of the previous cell is $\texttt{Library Loaded!}$. 

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd()+'grader')
from grader import workshop2 as ws2
from grader import practice

grader = ws2.Workshop2()

Last week's workshop was primarily about kinematics including concepts of *deformation gradient tensor* and *strains*.

This week we will look at *stresses* and *governing equations* that are used to formulate problems in solid mechanics. 

We will use SymPy for the majority of questions in this workshop. If you need a refresher, take a look back at the examples from last week.

As before, let's first import the modules we need:

In [ ]:
# SymPy Library: Symbolic Python
import sympy as sym

# Tell sympy to print things nicely
sym.init_printing()

# Matplotlib for plotting
import matplotlib.pyplot as plt

## Question 1
We consider a cube stretched biaxially (along two of its axes) along $\mathbf{e}_1$, $\mathbf{e}_2$ with corresponding internal tractions $T_1, T_2$ in that direction.
### a)

Find the corresponding stress tensor in matrix form, $S_{ij}$:




In [ ]:
# Solution:
T_1, T_2 = sym.symbols('T_1 T_2')

# Construct the S matrix
S = ...

# Display it
S

In [ ]:
# HINTS AND SOLUTION
#grader.hint1a()
grader.check1a(S)

### b)

What is the form of the stress tensor under equi-biaxial extension?

In [ ]:
# Equi-biaxial means T_1 = T_2 = T
T = sym.symbols('T')

# Construct the S matrix
S_equi = ...

# Display it
S_equi

In [ ]:
# HINTS AND SOLUTION
#grader.hint1b()
grader.check1b(S_equi)

## Question 2

Consider a body $B$ with constant density $\rho > 0$ subject to a constant body force per unit volume $\mathbf{b} = [0, 0, −\rho g]$ . Suppose the Cauchy stress field in $B$ is given by

$$
\mathbf{S} =
\begin{bmatrix}
x_2 & x_3 &  0 \\
x_3 & x_1 &  0 \\
0   &   0 & \rho g x_3
\end{bmatrix}
$$

Show that $\mathbf{b}$ and $S$ are consistent with the local equillibrium equations.

First, let's define a function to compute the divergence of a matrix, defined as:

$$ 
\mathbf{\nabla S} =
\begin{bmatrix}
\frac{\partial S_{11}}{\partial x_1} + \frac{\partial S_{12}}{\partial x_2}
+ \frac{\partial S_{13}}{\partial x_3} \\
\frac{\partial S_{21}}{\partial x_1} + \frac{\partial S_{22}}{\partial x_2}
+ \frac{\partial S_{23}}{\partial x_3} \\
\frac{\partial S_{31}}{\partial x_1} + \frac{\partial S_{32}}{\partial x_2}
+ \frac{\partial S_{33}}{\partial x_3}
\end{bmatrix}
$$

In [ ]:
def tensor_divergence(S):
    result = sym.Matrix([0,0,0])    
    # YOUR CODE HERE
    raise NotImplementedError
    
    return result

Then let's use this function to evaluate our equillibrium expression:

In [ ]:
rho, g, x_1, x_2, x_3 = sym.symbols('rho g x_1 x_2 x_3')

b = ...
S = ...
div_S = tensor_divergence(S)

div_S + b

In [ ]:
# HINTS AND SOLUTION
#grader.hint2()
grader.check2(div_S, b)

## Question 3

We consider a 2D deformation given by 
\begin{aligned}
x_1 &= 1.0005 X_1 + 0.001 X_2\\
x_2 &= 1.0002 X_2\\
\end{aligned}

We limit our analysis to the 'small strain' case, and assume that the linear description of elasticity holds.

To answer this question, you'll need four functions:
1. The deformation gradient matrix (as in last workshop).
2. The infinitesimal strain matrix (as in last workshop).
3. The plane strain linear elasticity matrix.
4. The plane stress linear elasticity matrix.

### a)
Evaluate the strain matrix (in Voigt notation)

In [ ]:
def deformation_gradient(phi, variables):
    dims = len(variables) 
    F = sym.Matrix.zeros(dims)
    
    # YOUR CODE HERE
    raise NotImplementedError
    
    return F

def infinitesimal_strain(F):

    # YOUR CODE HERE
    raise NotImplementedError

    return epsilon
    

In [ ]:
from grader import tools

# Define Lagrangian coordinates
X_1, X_2 = sym.symbols('X_1 X_2')
variables = sym.Matrix([X_1,X_2])

# Define the deformation
x_1 = ...
x_2 = ...

# Wrap it up as a vector
phi = ...

# Compute the deformation gradient
F = deformation_gradient(phi,variables)

# Compute the infinitesimal strain matrix
eps = infinitesimal_strain(F)

# Convert to Voigt notation
voigt_eps = ...

voigt_eps

In [ ]:
# HINTS AND SOLUTION
#grader.hint3a()
grader.check3a(voigt_eps)

### b)

Compute stresses under both plane strain and plane stress conditions assuming isotropic elasticity defined by two elastic constants: the Young modulus $E =4$GPa and Poisson’s ratio $\nu = 0.35$.

Here, you'll need to use the definitions of the **plane strain matrix** and the **plane stress matrix** in Voigt notation. They're simple enough to write out by hand, but you can write a function to do this. 

In [ ]:
def plane_strain(E, nu):
    
    # YOUR CODE HERE
    raise NotImplementedError
    
    return C_pe

def plane_stress(E, nu):
    
    # YOUR CODE HERE
    raise NotImplementedError
    
    return C_ps

In [ ]:
E = 4.0E9
nu = 0.35

# Plane strain conditions
C_pe = plane_strain(E, nu)

# The stresses can be calculated from the linear elasticity tensor C_pe and epsilon
stresses_pe =  ...
stresses_pe

In [ ]:
E = 4.0E9
nu = 0.35

# Plane stress conditions
C_ps = plane_stress(E, nu)

# The stresses can be calculated from the linear elasticity tensor C_ps and epsilon
stresses_ps = ...
stresses_ps

In [ ]:
# HINTS AND SOLUTION
#grader.hint3b()
grader.check3b(stresses_pe, stresses_ps)

## Question 4 🌶️

In this question, we're going to derive a continuum equation of motion from first principles. This is a more involved question than the others, but it is interesting. *This question will not be examined, so feel free to skip it*.

The question is mathematically very simple, so don't let a lack of familiarity with tensor calculus get in your way.

Your task is to set up a system of ODEs and solve them using SymPy. However, if you want to do this by hand, feel free: just input your answer in the solution cell in **SymPy notation**.

Newton's second law is usually stated in the form

$$
m\mathbf{a} = \mathbf{F}.
$$

In the framework of continuum mechanics, this is replaced by the linear momentum balance law:

$$
\rho \frac{\text{D}\mathbf{v}}{\text{D}t} = \nabla \cdot \mathbf{S} + \rho \mathbf{b}
$$

In this question, we're going to construct the equations of motion for a very simple system.

In [ ]:
# derive the symbols we need
t = sym.symbols('t')
X_1 = sym.Function('X_1')(t)
X_2 = sym.Function('X_2')(t)
X_3 = sym.Function('X_3')(t)

### a)
The left hand side of the equation is the material derivative. It is essentially a derivative that follows the motion of a particle that you track. In order to calculate it, you need to obtain the velocity associated with a deformation field, which is given by the formula

$$
\mathbf{v} = \frac{\partial \mathbf{\varphi}(\mathbf{X}(t), t)}{\partial t}
$$

Find the velocity field of the deformation described by
$$
\begin{gather*}
x_1 = 4 X_1\\
x_2 = X_2\\
x_3 = X_3
\end{gather*}
$$

In [ ]:
v1 = ...
v2 = ...
v3 = ...
v1, v2, v3

In [ ]:
#grader.hint4a()
grader.check4a(v1, v2, v3)

### b)
The material derivative itself is a little more complicated than a normal time derivative: in addition to how the field varies with time, you need to know how it varies spatially. 

$$
\frac{\text{D}\mathbf{v}}{\text{D}t} = \frac{\partial \mathbf{v}}{\partial t} + (\nabla \mathbf{v}) \mathbf{v}
$$

Calculate the material derivative for the velocity field of the previous part. The answer will be quite simple.

In [ ]:
a1 = ...
a2 = ...
a3 = ...

a1, a2, a3

In [ ]:
#grader.hint4b()
grader.check4b(a1, a2, a3)

### c)

Given the following tractions, build the Cauchy stress field and calculate $\nabla \cdot \mathbf{S}$. 
$$
\begin{gather*}
T_1 = 5 \mathbf{e}_1\\
T_2 = 5X_2^2 \mathbf{e}_2\\
T_3 = -9X_3^2 \mathbf{e}_3
\end{gather*}
$$

In [ ]:
# Make a small change to your original tensor divergence function.
def tensor_divergence(S):
    
    result = sym.Matrix([0,0,0])
    
    #YOUR CODE HERE
    raise NotImplementedError
    
    return result

# Construct the S matrix
S = ...

# Calculate tensor divergence
divt = tensor_divergence(S)
divt

In [ ]:
#grader.hint4c()
grader.check4c(divt)

### d) 
We're going to do something interesting with the body force and set it to be a damping force (proportional to velocity). Let 

$$
\mathbf{b} = -\left[ \begin{matrix} v_1 \\ 2v_2 \\ 5v_3 \end{matrix} \right]
$$

Now that you have all the components of the momentum balance law, derive the differential equations governing the motion of a single point. You should have three differential equations, each representing the motion of one coordinate of $(X_1, X_2, X_3)$. Like before, $\rho=1$.

In [ ]:
# Define the body force
b = ...

# Define sympy expressions for each coordinate which sum to zero
funx = ...
funy = ...
funz = ...
funx,funy,funz

In [ ]:
#grader.hint4d()
grader.check4d(funx, funy, funz)

Now, use SymPy to solve your differential equations. You can do this using the `dsolve` function included in `SymPy`, which is provided for you in the cells below. Of course, you can also solve them by hand!

In [ ]:
sym.dsolve(funx)

In [ ]:
sym.dsolve(funy)

In [ ]:
sym.dsolve(funz)

There are of course some constants that will show up in your answer. These can be solved by setting initial conditions for the deformation and velocity fields.

## Results

Run the box below to check your progress.

In [ ]:
grader.results()